First, activate conda environment:
```bash
conda activate NGTR
```

In [ ]:
import os
# Go outside the src directory
os.chdir("../Image2Table_LLM")
current_dir = os.getcwd()
print("Current directory:", current_dir)

In [ ]:
DATA_DIR = "../data/images"
GT_HTML_DIR = "../data/labels/tables"
GT_INFO_DIR = "../data/labels/info"
HTML_DIR = "../data/tables/html"
OUTPUT_JSON_DIR = "../data/json"
TEMP_DIR = "../data/temp"
SCHEMA_PATH = "../data/schema/personbasicinfo.yaml"
LLM_MODEL = "ollama/llama3"

In [ ]:
from LLM import call_LLM
from LLM_key import llm_model
from parse import extract_HTML, format_td
from prompt import tsr_html_prompt, tsr_html_decomposition
from metric import TEDS

In [ ]:
def build_html_table_for_single_image(image_name, output_html_path=HTML_DIR):
    image_path = os.path.join(DATA_DIR, image_name)
    llm_html = call_LLM(image_path, prompt=tsr_html_prompt, model_name=llm_model, temperature=0)
    llm_html = extract_HTML(llm_html)

    from bs4 import BeautifulSoup
    llm_html = llm_html.replace("<table>", "<table border='1'>")
    soup = BeautifulSoup(llm_html, 'html.parser')
    with open(os.path.join(output_html_path, image_name+'.html'), 'w', encoding='utf-8') as f:
        f.write(soup.prettify())


if __name__ == "__main__":
    for file in os.listdir(DATA_DIR): 
        if not file.endswith(".jpg"): 
            continue 

        image_name = file
        build_html_table_for_single_image(image_name)

In [ ]:
from LLM import call_LLM
from LLM_key import llm_model
from parse import extract_HTML, format_td
from prompt import tsr_html_prompt, tsr_html_decomposition
from metric import TEDS

In [ ]:
image_path = "../data/images/NL-HaNA_2.10.50_45_0110.jpg"
image_name = os.path.basename(image_path)

In [ ]:
llm_html = call_LLM(image_path, prompt=tsr_html_prompt, model_name=llm_model, temperature=0)
llm_html = extract_HTML(llm_html)

In [ ]:
from bs4 import BeautifulSoup
llm_html = llm_html.replace("<table>", "<table border='1'>")
soup = BeautifulSoup(llm_html, 'html.parser')
with open(os.path.join("../data/tables/html", image_name+'.html'), 'w', encoding='utf-8') as f:
    f.write(soup.prettify())

In [ ]:
def calculate_TEDS(ground_truth_html, predicted_html):
    predicted_html = format_td(predicted_html)
    ground_truth_html = format_td(ground_truth_html)

    teds = TEDS(structure_only=False)
    teds_score = teds.evaluate(ground_truth_html, predicted_html)

    teds_struct = TEDS(structure_only=True)
    teds_struct_score = teds_struct.evaluate(ground_truth_html, predicted_html)
    
    print(f"TEDS: {teds_score:.4f}")
    print(f"TEDS-Struct: {teds_struct_score:.4f}")

    return teds_score, teds_struct_score

In [ ]:
with open(os.path.join("../data/labels/tables", image_name+'.html'), 'r', encoding='utf-8') as f:
    label_html = f.read()

calculate_TEDS(label_html, llm_html)


### Information Extraction

In [ ]:
# Go outside the src directory
os.chdir("..")
current_dir = os.getcwd()
print("Current directory:", current_dir)

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(llm_html, 'html.parser')

rows = soup.find_all('tr')

# --- Step 1: make a grid keeping track of rowspans ---
logical_rows = []
rowspans = {}  # {(col_idx): [remaining_rows, cell_content]}

for r_idx, tr in enumerate(rows):
    # Start with any carried over rowspans from previous rows
    current_row = []
    to_remove = []

    # Fill carried-down cells first
    for col_idx, (remaining, cell) in rowspans.items():
        current_row.append(cell)
        rowspans[col_idx][0] -= 1
        if rowspans[col_idx][0] <= 0:
            to_remove.append(col_idx)
    for col_idx in to_remove:
        del rowspans[col_idx]

    # Now add new cells from this row
    c_idx = 0
    for td in tr.find_all('td'):
        while any(k == c_idx for k in rowspans):  # skip columns occupied by carried cells
            c_idx += 1
        text = td.get_text(" ", strip=True)
        cell_id = td.get('id')
        r_idx = int(td.get('row', r_idx))
        c_idx = int(td.get('col', c_idx))
        rowspan = int(td.get('rowspan', 1))
        colspan = int(td.get('colspan', 1))

        cell_data = {
            'text': text,
            'id': cell_id,
            'row': r_idx,
            'col': c_idx,
            'rowspan': rowspan,
            'colspan': colspan
        }

        # Place cell in current row
        current_row.append(cell_data)

        # Store for future rows if rowspan > 1
        if rowspan > 1:
            rowspans[c_idx] = [rowspan - 1, cell_data]

        c_idx += colspan

    logical_rows.append(current_row)


In [ ]:
if __name__ == "__main__":
    pagexml_path = "../data/tables/pagexml/NL-HaNA_2.10.50_45_0110.jpg.xml"
    image_path = "../data/images/NL-HaNA_2.10.50_45_0110.jpg"
    output_image_path = "../data/tables/visualisations/NL-HaNA_2.10.50_45_0110_viz.jpg"

    visualizer = PageXMLVisualizer(pagexml_path, image_path)
    visualizer.parse_pagexml()
    annotated_image = visualizer.draw_visualisation()

    # Save the visualised image
    cv2.imwrite(output_image_path, annotated_image)
    print(f"Visualisation saved to {output_image_path}")

In [ ]:
import re
import json
from src.person_info_extraction import extract_info_regex, extract_info_LLM

persons = []

for i, row in enumerate(logical_rows):
    person = {}
        
    # print(generate_prompt(cells))
    person = json.loads(extract_info_LLM(row))
    if all(v['value']==None for v in person.values()):
        continue

    if person:
        persons.append(person)

unique_persons = {json.dumps(person, sort_keys=True) for person in persons}
unique_persons_list = [json.loads(p) for p in unique_persons]
        
json_obj = {"persons": unique_persons_list}
print(json.dumps(json_obj, indent=2, ensure_ascii=False))

In [ ]:
with open(f"data/json/{image_name}.json", "w", encoding='utf-8') as json_file:
    json.dump(json_obj, json_file, ensure_ascii=False, indent=2)#!/usr/bin/env python3


In [ ]:
from src.metrics import best_match_similarity
# from src.metrics import calculate_normalized_information_distance

with open(f"data/json/{image_name}.json", 'r', encoding='utf-8') as f:
    constructed_html = json.load(f)

with open(os.path.join("data/labels", image_name.replace('.jpg', '.json')), 'r', encoding='utf-8') as f:
    label_html = json.load(f)

# calculate_normalized_information_distance(constructed_html, label_html)
print(best_match_similarity(label_html.get("persons", []), constructed_html.get("persons", [])))


### KG construction



In [ ]:
# load json from file
with open(f"../data/json/{image_name}.json", "r", encoding="utf-8") as f:
    json_obj = json.load(f)

In [ ]:
# CONSTRUCT ASSERSION TRIPLES
from rdflib import Graph, ConjunctiveGraph, Namespace, URIRef, Literal, RDF

FOAF = Namespace("http://xmlns.com/foaf/0.1/")
EX = Namespace("http://example.org/ontology/")
PROV = Namespace("http://www.w3.org/ns/prov#")

cg = ConjunctiveGraph()
cg.bind("foaf", FOAF)
cg.bind("ex", EX)
cg.bind("prov", PROV)

# Mapping from json keys to RDF predicates
predicate_map = {
    "vader": EX.vader,
    "moeder": EX.moeder,
    "geboorte_datum": EX.geboorteDatum,
    "geboorte_plaats": EX.geboortePlaats,
    "laatste_woonplaats": EX.laatsteWoonplaats
}

for idx, person in enumerate(json_obj["persons"], start=1):
    person_uri = URIRef(f"http://example.org/person/{idx}")
    assertion_graph_uri = URIRef("http://example.org/assertion")
    assertion_graph = Graph(store=cg.store, identifier=assertion_graph_uri)
    assertion_graph.add((person_uri, RDF.type, FOAF.Person))

    provenance_graph_uri = URIRef("http://example.org/provenance")
    provenance_graph = Graph(store=cg.store, identifier=provenance_graph_uri)

    for key, value_dict in person.items():
        value = value_dict["value"]
        cell_id = value_dict["cell"]
        
        predicate = predicate_map.get(key)
        
        # if cell_id is null, no named graph can be created
        if not cell_id:
            assertion_graph.add((person_uri, predicate, Literal(value)))
            continue
        
        if predicate:
            # Named graph for each cell
            graph_uri = URIRef(f"http://example.org/graph/{cell_id}")
            ng = Graph(store=cg.store, identifier=graph_uri)
            ng.add((person_uri, predicate, Literal(value)))
            

cg.serialize(f"../data/triples/{image_name}_assersion.trig", format='trig')
